In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import matplotlib.pyplot as plt
import pickle
import os

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print('cuda')
else:
    device = torch.device('cpu')
    print('cpu')
train_flag = True

In [ ]:
df = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
print(df.columns)
print(len(df))


In [ ]:
Y = df[['label']].values
X = df.drop(['label'], axis=1).values
print(len(X))
print(len(Y))
print(X.shape)
print(Y.shape)

In [ ]:
if train_flag:
    class Net(nn.Module):
        def __init__(self):
            super().__init__()
            self.fc1 = nn.Linear(784, 512)
            self.fc2 = nn.Linear(512, 1024)
            self.fc3 = nn.Linear(1024, 2048)
            self.fc4 = nn.Linear(2048, 10)
        def forward(self, x):
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = F.relu(self.fc3(x))
            x = F.softmax(self.fc4(x), dim=1)
            return x

    model = Net().to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    loss_function = nn.MSELoss()
    EPOCHS = 16
    batch_size = 64
    

In [ ]:
validate_perc = 0.1
validate_size = int(X.shape[0] * validate_perc)
X = X/255.0
Y = [np.eye(10)[i] for i in Y]
X = torch.Tensor(X)
Y = torch.Tensor(Y)
Y = Y.view(-1, Y.shape[2])
X_test = X[-validate_size:]
Y_test = Y[-validate_size:]
X = X[:-validate_size]
Y = Y[:-validate_size]

In [ ]:
if train_flag:
    losses = []
    for epoch in range(EPOCHS):
        losses_epoch = []
        for batch in tqdm(range(0, X.shape[0], batch_size)):
            x_batch = X[batch:batch+batch_size].view(-1, X.shape[1]).to(device)
            y_batch = Y[batch:batch+batch_size].to(device)
            model.zero_grad()
            y_hat = model.forward(x_batch)
            loss = loss_function(y_hat, y_batch)
            loss.backward()
            optimizer.step()
            if batch % 5*batch_size == 0:
                losses.append(loss)
                losses_epoch.append(loss)
        print(np.min(losses_epoch))

In [ ]:
if train_flag:    
    print(torch.save(model, 'model.h5'))
    plt.plot(losses)

In [ ]:
def validate(x, y):
    model = torch.load('model.h5')
    correct = 0
    total = 0
    with torch.no_grad():

        for i in tqdm(range(len(x))):
            real_class = torch.argmax(y[i].to(device))
            net_out = model.forward(x[i].view(-1, x.shape[1]).to(device))
            predicted_class = torch.argmax(net_out)

            if predicted_class == real_class:
                correct += 1
            total += 1
    print("Accuracy: ", round(correct/total, 3))
validate(X, Y)
validate(X_test, Y_test)

In [ ]:
df = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
print(df.columns)
X_test = df.values
print(len(X_test))
X_test = torch.Tensor(X_test)
X_test /= 255.0

In [ ]:
def test(x):
    model = torch.load('model.h5')
    with torch.no_grad():
        y = []
        for i in tqdm(range(len(x))):
            net_out = model.forward(x[i].view(-1, x.shape[1]).to(device))
            predicted_class = torch.argmax(net_out)
            y.append(int(predicted_class))
    return y
            
y = test(X_test)   


In [ ]:
print(len(y))
print(int(y[0]))

In [ ]:
df = pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv')
df['Label'] = 0
df['Label'] = y

In [ ]:
df.head()

In [ ]:
df.to_csv('result.csv', index=False)